# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 15 2022 9:02AM,253156,10,0086249499,ISDIN Corporation,Released
1,Dec 15 2022 9:02AM,253156,10,0086249504,ISDIN Corporation,Released
2,Dec 15 2022 9:02AM,253156,10,0086249495,ISDIN Corporation,Released
3,Dec 15 2022 9:02AM,253156,10,0086249494,ISDIN Corporation,Released
4,Dec 15 2022 9:02AM,253156,10,0086249528,ISDIN Corporation,Released
5,Dec 15 2022 9:02AM,253156,10,0086249529,ISDIN Corporation,Released
6,Dec 15 2022 9:02AM,253156,10,0086249536,ISDIN Corporation,Released
7,Dec 15 2022 9:02AM,253156,10,0086249535,ISDIN Corporation,Released
8,Dec 15 2022 9:02AM,253156,10,0086249542,ISDIN Corporation,Released
9,Dec 15 2022 9:02AM,253156,10,0086249537,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,253154,Released,1
30,253155,Released,1
31,253156,Cancelled,1
32,253156,Released,29
33,253158,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
253153,NaN,NaN,NaN,1.0
253154,NaN,NaN,NaN,1.0
253155,NaN,NaN,NaN,1.0
253156,1.0,NaN,NaN,29.0
253158,NaN,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
253064,0.0,0.0,0.0,22.0
253083,0.0,0.0,1.0,12.0
253087,0.0,10.0,0.0,15.0
253089,0.0,0.0,1.0,0.0
253090,0.0,0.0,0.0,43.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
253064,0.0,0,0,22
253083,0.0,0,1,12
253087,0.0,10,0,15
253089,0.0,0,1,0
253090,0.0,0,0,43


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,253064,0.0,0,0,22
1,253083,0.0,0,1,12
2,253087,0.0,10,0,15
3,253089,0.0,0,1,0
4,253090,0.0,0,0,43


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,253064,0.0,,,22
1,253083,0.0,,1,12
2,253087,0.0,10,,15
3,253089,0.0,,1,
4,253090,0.0,,,43


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 15 2022 9:02AM,253156,10,ISDIN Corporation
30,Dec 15 2022 8:51AM,253158,10,Emerginnova
33,Dec 15 2022 8:42AM,253155,16,Sartorius Bioprocess Solutions
34,Dec 15 2022 8:37AM,253154,19,"GCH Granules USA, Inc."
35,Dec 15 2022 8:19AM,253153,17,"Brookfield Pharmaceuticals, LLC"
36,Dec 15 2022 7:40AM,253149,19,"AdvaGen Pharma, Ltd"
67,Dec 15 2022 7:26AM,253146,22,"NBTY Global, Inc."
68,Dec 14 2022 4:48PM,253142,10,SugarBear
69,Dec 14 2022 3:59PM,253139,20,"Granules Pharmaceuticals, Inc."
70,Dec 14 2022 3:37PM,253138,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Dec 15 2022 9:02AM,253156,10,ISDIN Corporation,1.0,,,29
1,Dec 15 2022 8:51AM,253158,10,Emerginnova,0.0,,,3
2,Dec 15 2022 8:42AM,253155,16,Sartorius Bioprocess Solutions,0.0,,,1
3,Dec 15 2022 8:37AM,253154,19,"GCH Granules USA, Inc.",0.0,,,1
4,Dec 15 2022 8:19AM,253153,17,"Brookfield Pharmaceuticals, LLC",0.0,,,1
5,Dec 15 2022 7:40AM,253149,19,"AdvaGen Pharma, Ltd",0.0,,,31
6,Dec 15 2022 7:26AM,253146,22,"NBTY Global, Inc.",0.0,,1,
7,Dec 14 2022 4:48PM,253142,10,SugarBear,0.0,,,1
8,Dec 14 2022 3:59PM,253139,20,"Granules Pharmaceuticals, Inc.",0.0,,,1
9,Dec 14 2022 3:37PM,253138,19,"GCH Granules USA, Inc.",0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 9:02AM,253156,10,ISDIN Corporation,29,,
1,Dec 15 2022 8:51AM,253158,10,Emerginnova,3,,
2,Dec 15 2022 8:42AM,253155,16,Sartorius Bioprocess Solutions,1,,
3,Dec 15 2022 8:37AM,253154,19,"GCH Granules USA, Inc.",1,,
4,Dec 15 2022 8:19AM,253153,17,"Brookfield Pharmaceuticals, LLC",1,,
5,Dec 15 2022 7:40AM,253149,19,"AdvaGen Pharma, Ltd",31,,
6,Dec 15 2022 7:26AM,253146,22,"NBTY Global, Inc.",,1,
7,Dec 14 2022 4:48PM,253142,10,SugarBear,1,,
8,Dec 14 2022 3:59PM,253139,20,"Granules Pharmaceuticals, Inc.",1,,
9,Dec 14 2022 3:37PM,253138,19,"GCH Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 9:02AM,253156,10,ISDIN Corporation,29,,
1,Dec 15 2022 8:51AM,253158,10,Emerginnova,3,,
2,Dec 15 2022 8:42AM,253155,16,Sartorius Bioprocess Solutions,1,,
3,Dec 15 2022 8:37AM,253154,19,"GCH Granules USA, Inc.",1,,
4,Dec 15 2022 8:19AM,253153,17,"Brookfield Pharmaceuticals, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 9:02AM,253156,10,ISDIN Corporation,29.0,NaN,NaN
1,Dec 15 2022 8:51AM,253158,10,Emerginnova,3.0,NaN,NaN
2,Dec 15 2022 8:42AM,253155,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
3,Dec 15 2022 8:37AM,253154,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
4,Dec 15 2022 8:19AM,253153,17,"Brookfield Pharmaceuticals, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 9:02AM,253156,10,ISDIN Corporation,29.0,0.0,0.0
1,Dec 15 2022 8:51AM,253158,10,Emerginnova,3.0,0.0,0.0
2,Dec 15 2022 8:42AM,253155,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
3,Dec 15 2022 8:37AM,253154,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
4,Dec 15 2022 8:19AM,253153,17,"Brookfield Pharmaceuticals, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2531132,93.0,1.0,10.0
15,1012405,47.0,0.0,0.0
16,506280,2.0,0.0,0.0
17,253153,1.0,0.0,0.0
19,1771951,59.0,2.0,0.0
20,759363,2.0,1.0,0.0
21,253132,1.0,0.0,0.0
22,253146,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2531132,93.0,1.0,10.0
1,15,1012405,47.0,0.0,0.0
2,16,506280,2.0,0.0,0.0
3,17,253153,1.0,0.0,0.0
4,19,1771951,59.0,2.0,0.0
5,20,759363,2.0,1.0,0.0
6,21,253132,1.0,0.0,0.0
7,22,253146,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,93.0,1.0,10.0
1,15,47.0,0.0,0.0
2,16,2.0,0.0,0.0
3,17,1.0,0.0,0.0
4,19,59.0,2.0,0.0
5,20,2.0,1.0,0.0
6,21,1.0,0.0,0.0
7,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,93.0
1,15,Released,47.0
2,16,Released,2.0
3,17,Released,1.0
4,19,Released,59.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,19,20,21,22
Status,,,,,,,,
Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,1.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0
Released,93.0,47.0,2.0,1.0,59.0,2.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,19,20,21,22
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,1.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0
2,Released,93.0,47.0,2.0,1.0,59.0,2.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,19,20,21,22
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,1.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0
2,Released,93.0,47.0,2.0,1.0,59.0,2.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()